In [2]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random

ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]

In [3]:
# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy(proxies):
    return random.randint(0, len(proxies) - 1)

def setup_proxy_rotation():
    # Retrieve latest proxies
    proxies_req = Request('https://www.sslproxies.org/')
    proxies_req.add_header('User-Agent', ua.random)
    proxies_doc = urlopen(proxies_req).read().decode('utf8')

    soup = BeautifulSoup(proxies_doc, 'html.parser')
    proxies_table = soup.find(id='proxylisttable')

    # Save proxies in the array
    for row in proxies_table.tbody.find_all('tr'):
        proxies.append({'ip':   row.find_all('td')[0].string,
                        'port': row.find_all('td')[1].string })
    return proxies

proxies = setup_proxy_rotation()
# Choose a random proxy
proxy_index = random_proxy(proxies)
proxy = proxies[proxy_index]

for n in range(1, 2):
    req = Request('http://icanhazip.com')
    req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')
    print('its not this..')

    # Every 10 requests, generate a new proxy
    if n % 30 == 0:
        proxy_index = random_proxy(proxies)
        proxy = proxies[proxy_index]

    # Make the call
    try:
        print('is it this??')
        my_ip = urlopen(req).read().decode('utf8')
        print('#' + str(n) + ': ' + my_ip)
    except: # If error, delete this proxy and find another one
        del proxies[proxy_index]
        print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
        proxy_index = random_proxy(proxies)
        proxy = proxies[proxy_index]

its not this..
is it this??
Proxy 177.103.223.155:3128 deleted.


In [4]:
req